# Music Genre Classification

**Aidan O'Keefe**

PICTURE OF AUDIO FILE SPECTROGRAM GOES HERE

<br>

## Overview

OVERVIEW HERE


## Business Understanding

Business Case 1: An app for music recogniztion does not just want to find the song of that you are hearing (like Shazam and SoundHound do) but also wants to find the genre so it can then better inform its recommendations. 

Business Case 2: A Folk Music researcher (musicologist) is recording cultural music in the field and wants to know what genre to categorize his recordings. He think that maybe by finding what modern genre a recording is classified as will help explain which modern genres are influenced by traditional folk music. (Jazz being influenced by African rhythms and Bluegrass from traditional Irish music). In order to help the musicologist, I am extracting data from audio files and building a model which can classify a recording into a genre based on the data in its audio file.

## Data Understanding

I chose to use "GTZAN Dataset - Music Genre Classification" from Kaggle which is a dataset of 1000 audio tracks of a 30-second duration. The 1000 rows of data are evenly divided into 10 genres with 100 tracks each.  

Each row represents a 30-second long song recording. The columns contain a mean and variance computed over multiple features that can be extracted from an audio file.

In [1]:
#Decription of each column feature

### Importing The Data

In [5]:
#Import standard libraries
import numpy as np
import pandas as pd

#Import Librosa for audio to text creation
# import librosa.core
# import librosa.display

In [6]:
#Load the data csv
music_df = \
pd.read_csv("/Users/Aidan/Documents/Flatiron/Phase_3/Phase_3_project/Music_Genre_Classification/data/features_30_sec.csv") 

In [7]:
#Preview the first five records
music_df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,blues
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,blues
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,blues
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,blues
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,blues


### Exploring The Data

In [8]:
#Look at our datatypes and for any missing values
music_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 60 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   filename                 1000 non-null   object 
 1   length                   1000 non-null   int64  
 2   chroma_stft_mean         1000 non-null   float64
 3   chroma_stft_var          1000 non-null   float64
 4   rms_mean                 1000 non-null   float64
 5   rms_var                  1000 non-null   float64
 6   spectral_centroid_mean   1000 non-null   float64
 7   spectral_centroid_var    1000 non-null   float64
 8   spectral_bandwidth_mean  1000 non-null   float64
 9   spectral_bandwidth_var   1000 non-null   float64
 10  rolloff_mean             1000 non-null   float64
 11  rolloff_var              1000 non-null   float64
 12  zero_crossing_rate_mean  1000 non-null   float64
 13  zero_crossing_rate_var   1000 non-null   float64
 14  harmony_mean             

It looks like we have no missing values and all of our features are numerical except for file name and label.

### Visualizing the Data

In [16]:
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats as stats

%matplotlib inline

## Data Preparation

In [11]:
#LabelEncoder to turn our target classes into numbers
from sklearn.preprocessing import LabelEncoder

#Instantiate our LabelEncoder
lab_enc = LabelEncoder()

#Fit and Transform our target variable and replace the 'label' column in the dataset
music_df['label'] = lab_enc.fit_transform(music_df['label'])

#Check the 'label' column is now encoded
music_df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.wav,661794,0.350088,0.088757,0.130228,0.002827,1784.165850,129774.064525,2002.449060,85882.761315,...,52.420910,-1.690215,36.524071,-0.408979,41.597103,-2.303523,55.062923,1.221291,46.936035,0
1,blues.00001.wav,661794,0.340914,0.094980,0.095948,0.002373,1530.176679,375850.073649,2039.036516,213843.755497,...,55.356403,-0.731125,60.314529,0.295073,48.120598,-0.283518,51.106190,0.531217,45.786282,0
2,blues.00002.wav,661794,0.363637,0.085275,0.175570,0.002746,1552.811865,156467.643368,1747.702312,76254.192257,...,40.598766,-7.729093,47.639427,-1.816407,52.382141,-3.439720,46.639660,-2.231258,30.573025,0
3,blues.00003.wav,661794,0.404785,0.093999,0.141093,0.006346,1070.106615,184355.942417,1596.412872,166441.494769,...,44.427753,-3.319597,50.206673,0.636965,37.319130,-0.619121,37.259739,-3.407448,31.949339,0
4,blues.00004.wav,661794,0.308526,0.087841,0.091529,0.002303,1835.004266,343399.939274,1748.172116,88445.209036,...,86.099236,-5.454034,75.269707,-0.916874,53.613918,-4.404827,62.910812,-11.703234,55.195160,0


In [12]:
#Define our predictive variables and target variable
X = music_df.drop(['label','filename'], axis=1)
y = music_df['label']

Dropped both non-numeric columns from our X as `label` is our target variable and `filename` is just an id and not intrinsic to the data.

In [13]:
#Split the data to prevent data leakage using a 80/20 train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
train_test_split(X, y, stratify = y, test_size = 0.2, random_state= 10)

In [14]:
# Let's look to see if our target classes are balanced
y_train.value_counts()

9    80
8    80
7    80
6    80
5    80
4    80
3    80
2    80
1    80
0    80
Name: label, dtype: int64

## Modeling

In [28]:
#Classification algorithm
from sklearn.linear_model import LogisticRegression

#Pipeline 
from sklearn.pipeline import Pipeline

#Scalers
from sklearn.preprocessing import StandardScaler, MaxAbsScaler, Normalizer

### Baseline Model

In [39]:
#Create variable storing our scaler and model
steps = [('stdscale', StandardScaler()), ('LR', LogisticRegression(solver= "liblinear", random_state= 10))]

#Instantiate the Modeling Pipeline
model = Pipeline(steps)

#Fit to our training data
model.fit(X_train, y_train)

Pipeline(steps=[('stdscale', StandardScaler()),
                ('LR',
                 LogisticRegression(random_state=10, solver='liblinear'))])

In [43]:
# Let's see our baseline train and test scores
print('Train baseline:', model.score(X_train, y_train))

print('Test baseline:', model.score(X_test, y_test))

Train baseline: 0.83875
Test baseline: 0.73


### Second Model

In [ ]:
#Classification algorithms
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

#Crossvalidation packages
from sklearn.model_selection import  GridSearchCV, cross_val_score, RandomizedSearchCV

In [ ]:
#Preprocessing Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

#Create preprocessing pipeline for X variables
# preprocess_pipe = Pipeline([('stdscale', StandardScaler())])

In [ ]:
#List the estimators we want to use
# estimators= [('LR', LogisticRegression(random_state= 10))]

#Create variable storing preprocessing pipeline and our estimators
# steps = [('stdscale', StandardScaler()), ('LR', LogisticRegression(solver= "liblinear", random_state= 10))]

#Create list of hyperparameters to try
# params = {'estimators__LR__penalty': ['l1', 'l2'],
#          'estimators__LR__solver': ['liblinear', 'saga'],
#          'estimators__LR__C': [.1, 1.0, 10, 100, 1000]}

#Instantiate the Modeling Pipeline
# model = Pipeline(steps)

#5 Fold Randomized Cross Validations
# rand_cv = RandomizedSearchCV(estimator = model, param_distributions = params,\
#                              cv = 5, random_state=10)

# rand_cv.fit(X_train, y_train)

In [ ]:
# Let's see our best score
# rand_cv.best_score_

In [ ]:
#Let's view our top 5 results as a dataframe
rand_cv_results = pd.DataFrame(rand_cv.cv_results_)
rand_cv_results.head()

In [ ]:
#Save our model with the best parameters
best_model = rand_cv.best_estimator_

In [ ]:
#Fit our train data on our best model
best_model.fit(X_train, y_train)

In [ ]:
#Predict on our Test data
y_pred = best_model.predict(X_test)

## Evaluation

In [ ]:
from sklearn.metrics import precision_score, recall_score, plot_confusion_matrix

In [ ]:
#Final model evaluation 

In [ ]:
print(classification_report(y_test, y_pred))

Explanation of results

### Conclusion

What did we learn from this project. How do we hope to implement this for our business case